In [223]:
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from keras.models import Sequential, model_from_json
from keras.layers.merge import concatenate
from keras.layers.core import Dropout, Dense, Activation, Flatten, Reshape
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from collections import defaultdict
from sklearn import preprocessing

In [263]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="embeddings/glove/glove.twitter.27B.50d.txt", word2vec_output_file="embeddings/glove/gensim_glove.twitter.27B.50d.txt")

(1193517, 50)

In [264]:
glove_model = KeyedVectors.load_word2vec_format("embeddings/glove/gensim_glove.twitter.27B.50d.txt", binary=False)

In [265]:
with open('data/train.jsonl') as f:
    train_data = pd.read_json(f, lines=True)
with open('data/test.jsonl') as f:
    test_data = pd.read_json(f, lines=True)

In [266]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(train_data['label'])
y_train = np_utils.to_categorical(y, num_classes=2)

In [267]:
train_data['context'] = train_data['context'].str.join(' ')

In [270]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH_RESPONSE = 100
MAX_SEQUENCE_LENGTH_CONTEXT = 200
MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH_RESPONSE + MAX_SEQUENCE_LENGTH_CONTEXT

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower = False)
tokenizer.fit_on_texts(train_data['response'])
tokenizer.fit_on_texts(train_data['context'])
tokenizer.fit_on_texts(test_data['response'])
tokenizer.fit_on_texts(test_data['context'])
X_train_resp = pad_sequences(tokenizer.texts_to_sequences(train_data['response']), maxlen=MAX_SEQUENCE_LENGTH_RESPONSE)
X_train_cont = pad_sequences(tokenizer.texts_to_sequences(train_data['context']), maxlen=MAX_SEQUENCE_LENGTH_CONTEXT)
#X_train = np.hstack((X_train_resp, X_train_cont))
X_test_resp = pad_sequences(tokenizer.texts_to_sequences(test_data['response']), maxlen=MAX_SEQUENCE_LENGTH_RESPONSE)
X_test_cont = pad_sequences(tokenizer.texts_to_sequences(test_data['context']), maxlen=MAX_SEQUENCE_LENGTH_CONTEXT)
#X_test = np.hstack((X_test_resp, X_test_cont))

In [271]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 46848 unique tokens.


In [272]:
EMBEDDING_DIM = glove_model.vector_size

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in glove_model.vocab:
        embedding_matrix[i] = glove_model[word]

In [273]:
batch_size = 8
input_response = keras.Input(batch_shape=(batch_size, MAX_SEQUENCE_LENGTH_RESPONSE), dtype="int32")
embed_response = layers.Embedding(len(word_index) + 1,
                          EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=MAX_SEQUENCE_LENGTH_RESPONSE,
                          trainable=False)(input_response)
input_context = keras.Input(batch_shape=(batch_size, MAX_SEQUENCE_LENGTH_CONTEXT), dtype="int32")
embed_context = layers.Embedding(len(word_index) + 1,
                          EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=MAX_SEQUENCE_LENGTH_CONTEXT,
                          trainable=False)(input_context)
x = layers.Concatenate(axis = 1)([embed_response ,embed_context])
x = layers.Conv1D(EMBEDDING_DIM, 3, kernel_initializer='he_normal', padding='valid',
                        activation='sigmoid')(x)
x = layers.Conv1D(EMBEDDING_DIM, 3, kernel_initializer='he_normal', padding='valid',
                        activation='sigmoid')(x)
x = layers.Dropout(0.25)(x)
x = layers.LSTM(128, kernel_initializer='he_normal', activation='sigmoid', dropout=0.5,
               return_sequences=True)(x)
x = layers.LSTM(128, kernel_initializer='he_normal', activation='sigmoid', dropout=0.5)(x)
x = layers.Dense(128, kernel_initializer='he_normal', activation='sigmoid')(x)
predictions = layers.Dense(2, activation='softmax')(x)
model = keras.Model([input_response, input_context], predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('No of parameter:', model.count_params())
print(model.summary())

No of parameter: 4940002
Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_59 (InputLayer)           [(8, 100)]           0                                            
__________________________________________________________________________________________________
input_60 (InputLayer)           [(8, 200)]           0                                            
__________________________________________________________________________________________________
embedding_82 (Embedding)        (8, 100, 50)         2342450     input_59[0][0]                   
__________________________________________________________________________________________________
embedding_83 (Embedding)        (8, 200, 50)         2342450     input_60[0][0]                   
_____________________________________________________________

In [ ]:
modelCheckpoint = ModelCheckpoint('best_sarcasm_model.hdf5', save_best_only = True)
model.fit([X_train_resp, X_train_cont], y_train, batch_size=batch_size, epochs=100, validation_split=0.2 , shuffle=True, callbacks=[modelCheckpoint])

Epoch 1/100
500/500 [==============================] - 64s 127ms/step - loss: 0.6698 - accuracy: 0.6170 - val_loss: 1.1059 - val_accuracy: 0.0000e+00
Epoch 2/100
500/500 [==============================] - 65s 130ms/step - loss: 0.6709 - accuracy: 0.6125 - val_loss: 1.2645 - val_accuracy: 0.0000e+00
Epoch 3/100
500/500 [==============================] - 69s 139ms/step - loss: 0.6606 - accuracy: 0.6152 - val_loss: 0.9823 - val_accuracy: 0.2760
Epoch 4/100
500/500 [==============================] - 70s 140ms/step - loss: 0.6059 - accuracy: 0.6867 - val_loss: 1.1443 - val_accuracy: 0.2050
Epoch 5/100
500/500 [==============================] - 70s 140ms/step - loss: 0.5751 - accuracy: 0.7147 - val_loss: 1.3995 - val_accuracy: 0.1560
Epoch 6/100
500/500 [==============================] - 69s 137ms/step - loss: 0.5662 - accuracy: 0.7262 - val_loss: 1.3640 - val_accuracy: 0.2120
Epoch 7/100
500/500 [==============================] - 67s 134ms/step - loss: 0.5535 - accuracy: 0.7275 - val_loss: 